# # 모듈
---

In [ ]:
import pandas as pd
import numpy as np
import json
import glob
import os
import pathlib
import re
import shutil
from sklearn.preprocessing import LabelEncoder
from PIL import Image, ImageDraw, ImageFont

# # NIA50_sample
---

In [ ]:
label_path = '/data/NIA50/data/2-050_sensor_sample/2d_label/'
front_labels = [word for word in os.listdir(label_path) if 'front' in word]
rear_labels = [word for word in os.listdir(label_path) if 'rear' in word]

In [ ]:
# 최신 sample
label_path = '/data/NIA50/data/sample/2d_label/'
rear_labels = os.listdir(label_path)

## # labels
---

### # front_label
---

In [ ]:
df_f2dl = pd.DataFrame(columns = ['x_min', 'y_min', 'x_max', 'y_max', 'id', 'class', 'filename'])

num = 0
for front_label in sorted(front_labels)[1:]:
    with open(label_path+front_label, 'rb') as f:
        front_js = json.load(f)
    
    for obj in front_js['objects']:
        
        x_point = []
        y_point = []
        for points in obj['front']:
            x_point.append(points['x'])
            y_point.append(points['y'])
            
        for points in obj['back']:
            x_point.append(points['x'])
            y_point.append(points['y'])
            
        x_min = min(x_point) / 1.6
        if x_min < 0: x_min = 0
        
        y_min = min(y_point) / 1.6
        if y_min < 0: y_min = 0
        
        x_max = max(x_point) / 1.6
        if x_max > 1280: x_max = 1280
        
        y_max = max(y_point) / 1.6
        if y_max > 960: y_max = 960
        
        id_ = obj['id']
        class_ = obj['class']
        filename = front_js['filename'][:-4]
        
        data = [x_min, y_min, x_max, y_max, id_, class_, filename]

        df_f2dl.loc[num] = data
        
        num+=1
        
df_f2dl['class'] = df_f2dl['class'].str.lower()
df_f2dl.loc[(df_f2dl['class'] == 'box-svg-selected') | (df_f2dl['class'] == 'medium_truck'), 'class'] = 'truck'
df_f2dl.loc[(df_f2dl['class'] == 'adult'), 'class'] = 'person'
df_f2dl.loc[(df_f2dl['class'] == 'suv'), 'class'] = 'car'
# df_f2dl['class'] = df_f2dl['class'].apply(lambda x: x.capitalize())
# df_f2dl['label'] = LabelEncoder().fit_transform(df_f2dl['class'])
df_f2dl.loc[df_f2dl['class'] == 'person', 'label'] = 0
df_f2dl.loc[df_f2dl['class'] == 'car', 'label'] = 2
df_f2dl.loc[df_f2dl['class'] == 'bus', 'label'] = 5
df_f2dl.loc[df_f2dl['class'] == 'truck', 'label'] = 7
df_f2dl['label'] = df_f2dl['label'].astype('int')

df_f2dl

In [ ]:
print(df_f2dl['class'].unique())
print(df_f2dl['label'].unique())

### # rear_label
---


In [ ]:
df_r2dl = pd.DataFrame(columns = ['x_min', 'y_min', 'x_max', 'y_max', 'id', 'class', 'filename'])

num = 0
for rear_label in sorted(rear_labels)[1:]:
    with open(label_path+rear_label, 'rb') as f:
        rear_js = json.load(f)
    
    for obj in rear_js['objects']:
        
        x_point = []
        y_point = []
        for points in obj['front']:
            x_point.append(points['x'])
            y_point.append(points['y'])
            
        for points in obj['back']:
            x_point.append(points['x'])
            y_point.append(points['y'])
            
        x_min = min(x_point) / 1.6
        if x_min < 0: x_min = 0
        
        y_min = min(y_point) / 1.6
        if y_min < 0: y_min = 0
        
        x_max = max(x_point) / 1.6
        if x_max > 1280: x_max = 1280
        
        y_max = max(y_point) / 1.6
        if y_max > 960: y_max = 960
        
        id_ = obj['id']
        class_ = obj['class']
        filename = rear_js['filename'][:-4]
        
        data = [x_min, y_min, x_max, y_max, id_, class_, filename]

        df_r2dl.loc[num] = data
        
        num+=1

df_r2dl

In [ ]:
print(df_r2dl['class'].unique())
# print(df_r2dl['label'].unique())

# # yolov5용 label 만들기
---

In [ ]:
def kitti_to_yolo(label, x1, y1, x2, y2, image_w, image_h): # x1=x_min, y1=y_min, x2=x_max, y2=y_max
    return [label, ((x2 + x1)/(2*image_w)), ((y2 + y1)/(2*image_h)), (x2 - x1)/image_w, (y2 - y1)/image_h]

In [ ]:
for filename in df_f2dl['filename'].unique():

    df_lb = pd.DataFrame(columns = ['class', 'x', 'y', 'width', 'height'])
    for i, j in enumerate(df_f2dl.loc[df_f2dl['filename'] == filename].index):
        tmp = df_f2dl.iloc[j]
        label_data = kitti_to_yolo(tmp['label'], tmp['x_min'], tmp['y_min'], tmp['x_max'], tmp['y_max'], 1280, 960)
        
        df_lb.loc[i] = label_data
        df_lb['class'] = df_lb['class'].astype('int')
        
    df_lb = df_lb.loc[df_lb['width'] > 0]
    df_lb = df_lb.loc[df_lb['height'] > 0]
    df_lb.to_csv(f'/data/NIA50/data/2-050_sensor_sample/camera/front/labels/{filename}.txt', index = None, header = None, sep = ' ')

## # front
---

In [ ]:
from PIL import Image, ImageDraw, ImageFont

font_size = 15
color = (0, 255, 0)

for j in df_f2dl['filename'].unique()[:1]:
    img = f'/data/NIA50/data/2-050_sensor_sample/camera/front/images/{j}.jpg'
    img_ = Image.open(img).convert('RGB')
    
    temp = df_f2dl.loc[df_f2dl['filename'] == j]
    for i in temp.index:
        data = temp.loc[i]
        
        # color = tuple(np.random.choice(255, 3))
        font = ImageFont.truetype('/data/NIA50/kimgh/fonts/arial.ttf', 18) # arial.ttf 글씨체, font_size=15
        # font = ImageFont.load_default()
        # box_color_RGBA  = (0,255,0,255)
        # fill_color_RGBA = (0,255,0,50)
        # draw = ImageDraw.Draw(img_, 'RGBA') # RGBA
        # draw.rectangle((100,100,300,300), outline=box_color_RGBA, fill=fill_color_RGBA, width = 3)
        # img_.show()

        draw = ImageDraw.Draw(img_)
        text_pos = (data[0], data[1]-23)
        draw.text(text_pos, f"{data['id'], data['class']}", fill = color, font = font, stroke_width = 1)
        draw.rectangle(list(data[:4]), outline = color, width = 3)
    
    print('filename :', j)    
    img_.show()

## # rear
---

In [ ]:
from PIL import Image, ImageDraw, ImageFont

font_size = 15
color = (0, 255, 0)

for j in df_r2dl['filename'].unique()[90:101]:
    img = f'/data/NIA50/data/sample/camera/rear/{j}.jpg'
    img_ = Image.open(img).convert('RGB')
    
    temp = df_r2dl.loc[df_r2dl['filename'] == j]
    for i in temp.index:
        data = temp.loc[i]
        
        # color = tuple(np.random.choice(255, 3))
        font = ImageFont.truetype('/data/NIA50/kimgh/fonts/arial.ttf', 18) # arial.ttf 글씨체, font_size=15
        # font = ImageFont.load_default()
        # box_color_RGBA  = (0,255,0,255)
        # fill_color_RGBA = (0,255,0,50)
        # draw = ImageDraw.Draw(img_, 'RGBA') # RGBA
        # draw.rectangle((100,100,300,300), outline=box_color_RGBA, fill=fill_color_RGBA, width = 3)
        # img_.show()

        draw = ImageDraw.Draw(img_)
        text_pos = (data[0], data[1]-23)
        draw.text(text_pos, f"{data['id'], data['class']}", fill = color, font = font, stroke_width = 1)
        draw.rectangle(list(data[:4]), outline = color, width = 3)
        
    print('filename :', j)    
    img_.show()

# # NIA48
---

## # train, val, test
---

In [ ]:
with open('/data/NIA50/50-2/data/NIA48/temp_data/ImageSets/train.txt', 'r') as f:
    train = [i.replace('\n', '') for i in f.readlines()]
    
with open('/data/NIA50/50-2/data/NIA48/temp_data/ImageSets/val.txt', 'r') as f:
    val = [i.replace('\n', '') for i in f.readlines()]

In [ ]:
# import glob
# from sklearn.model_selection import train_test_split

# img_list = sorted(glob.glob('/data/NIA50/data/yolo_aivill48/images/*.png'))

# train_list, val_list = train_test_split(img_list, test_size = 0.2, random_state = 0)

# with open('/data/NIA50/data/yolo_aivill48/train.txt', 'w') as f:
#     f.write('\n'.join(train_list) + '\n')
    
# with open('/data/NIA50/data/yolo_aivill48/val.txt', 'w') as f:
#     f.write('\n'.join(val_list) + '\n')

## # images
---

In [50]:
save_dir = '/data/NIA50/50-2/data/NIA48/temp_data/ImageSets/'
os.makedirs(save_dir, exist_ok=True)

image_path = '/data/NIA50/50-2/data/NIA48/temp_data/images/'

train_path = [(image_path + i + '.png') for i in train]
with open(save_dir + 'train_yolo.txt', 'w') as f:
    f.write('\n'.join(train_path))
    
val_path = [(image_path + i + '.png') for i in val]
with open(save_dir + 'val_yolo.txt', 'w') as f:
    f.write('\n'.join(val_path))

In [ ]:
# # 이미지 복사

# import os
# import shutil


# path = '/data/NIA50/data/yolo_aivill48/'

# for i in avante_labels:
#     os.makedirs(path+i, exist_ok=True)
 
    
# all_path = '/data/NIA50/data/Avante_Clips/*/*'

# paths = []
# for i in glob.glob(all_path):
#     for j in avante_labels:
#         if j in i:
#             paths.append(i)
            

# # 폴더 생성
# for path in paths:
#     shutil.copytree(path + '/Camera/CameraFront/blur', '/data/NIA50/data/yolo_aivill48/' + path[-15:], dirs_exist_ok=True)

## # labels 
---

In [14]:
def xywh2xyxy(x):
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    x = np.array(x).reshape(1, -1)
    y = np.copy(x)
    y[:, 0] = x[:, 0] - x[:, 2] / 2  # top left x
    y[:, 1] = x[:, 1] - x[:, 3] / 2  # top left y
    y[:, 2] = x[:, 0] + x[:, 2] / 2  # bottom right x
    y[:, 3] = x[:, 1] + x[:, 3] / 2  # bottom right y
    y = list(y.reshape(-1))
    return y


def xyxy2xywhn(x, w=1920, h=1200, clip=False, eps=0.0):
    # Convert nx4 boxes from [x1, y1, x2, y2] to [x, y, w, h] normalized where xy1=top-left, xy2=bottom-right
    if clip:
        clip_boxes(x, (h - eps, w - eps))  # warning: inplace clip
    x = np.array(x).reshape(1, -1)
    y = np.copy(x)
    y[:, 0] = ((x[:, 0] + x[:, 2]) / 2) / w  # x center
    y[:, 1] = ((x[:, 1] + x[:, 3]) / 2) / h  # y center
    y[:, 2] = (x[:, 2] - x[:, 0]) / w  # width
    y[:, 3] = (x[:, 3] - x[:, 1]) / h  # height
    y = list(y.reshape(-1))
    return y

In [35]:
import json
import re

save_dir = '/data/NIA50/50-2/data/NIA48/temp_data/labels_yolo/'
os.makedirs(save_dir, exist_ok=True)

# {MOTORCYCLE: [BICYCLE, MOTORCYCLE], CAR: [CAR, BUS, TRUCK, ETC], PEDESTRIAN: PEDESTRIAN}
class_ = {'CAR': 0, 'BUS': 0, 'TRUCK': 0, 'ETC': 0, 'BIBYCLE': 1, 'MOTORCYCLE': 1, 'PEDESTRIAN': 2}

label_path = '/data/NIA50/50-2/data/NIA48/temp_data/labels_json/'
# labels = glob.glob(label_path+'*.json')
labels = sorted(os.listdir(label_path))

for label in labels:                     
    with open(label_path+label, 'r') as f:
        label_js = json.load(f)
        name = re.sub('FC_', '', label[:-5])
        f.close()
        
        with open (save_dir + name + '.txt', 'w') as f:
            
            for i in np.arange(len(label_js['annotation'])):
                category = label_js['annotation'][i]['category']
                xywh = label_js['annotation'][i]['3d_box'][0]['2d_box']
                
                if xywh[0] < 0: xywh[0] = 0
                
                f.write(f"{class_[category]} {xywh[0]/1920} {xywh[1]/1200} {xywh[2]/1920} {xywh[3]/1200}\n")

            f.close()

In [ ]:
# path = '/data/NIA50/data/Avante_Clips/'
# avante_clip = []
# for i in os.listdir(path):
#     clip = os.listdir(path+i)
#     avante_clip+=clip

In [ ]:
# # 라벨 복사

# import glob
# import shutil
    
# sorted(avante_clip)
# all_path = glob.glob('/data/NIA50/data/aivill-48/*/*/label')

# avante_labels = []
# for path in all_path:
#     for i in os.listdir(path):
#         if 'A_Clip' in i:
#             # print(path + '/' + i)
#             shutil.copytree(path + '/' + i + '/result', '/data/NIA50/data/yolo_aivill48/' + i, dirs_exist_ok=True)
#             avante_labels.append(i)

# # 특수환경 자율주행 3D 이미지
---

In [ ]:
val_df = pd.read_csv('/data/NIA50/data/특수환경 자율주행 3D 이미지/val_df.csv')
val_df['frame'] = val_df['frame'].apply(lambda x: str(x).zfill(6))
tdf = val_df.loc[val_df['xyzlwh']!='[0.0, 0.0, 0.0, 0, 0, 0]']
tdf.head()

In [ ]:
tdf['class'].unique()

In [ ]:
temp = t_df.loc[t_df['xyxy'].apply(lambda x: '-' in x)]

In [ ]:
tt = json.loads(temp['xyxy'].values[0])

In [ ]:
img_ = Image.open('/data/NIA50/data/특수환경 자율주행 3D 이미지/Validation/003/1/drive_0009/image_0/daejeon_003_1_drive_0009_00000034.jpg').convert('RGB')


color = tuple(np.random.choice(255, 3))
font = ImageFont.truetype('/data/NIA50/kimgh/fonts/arial.ttf', 18) # arial.ttf 글씨체, font_size=15
# font = ImageFont.load_default()
# box_color_RGBA  = (0,255,0,255)
# fill_color_RGBA = (0,255,0,50)
# draw = ImageDraw.Draw(img_, 'RGBA') # RGBA
# draw.rectangle((100,100,300,300), outline=box_color_RGBA, fill=fill_color_RGBA, width = 3)
# img_.show()

draw = ImageDraw.Draw(img_)
# text_pos = (data[0], data[1]-23)
# draw.text(text_pos, f"{data['id'], data['class']}", fill = color, font = font, stroke_width = 1)
draw.rectangle(js['OBJECT_LIST'][0]['3D_LIST'][0]['BOX'], outline = color, width = 3)
img_
